# Comparing various values from ISIS and USGSCSM cameras

In [ ]:
import ale
from ale.drivers.messenger_drivers import MessengerMdisPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
import json
import os
from pysis import isis
import pvl
import numpy as np
import knoten
import csmapi
from knoten import csm

In [ ]:
# printing config displays the yaml formatted string
print(ale.config)

# config object is a dictionary so it has the same access patterns 
print('MDIS spice directory:', ale.config['mdis'])

# updating config for new MDIS path in this notebook 
# Note: this will not change the path in `.ale/config.yml`. This change only lives in the notebook.
# ale.config['mdis'] = '/home/kdlee/builds/knoten'

In [ ]:
# Declare variables to use outside of with
j2000RotationPos = 0
j2000RotationVel = 0

# change to desired PDS3 image path 
fileName = '/home/kdlee/builds/ale/EN1072174528M.IMG'

# metakernels are furnsh-ed when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with MessengerMdisPds3NaifSpiceDriver(fileName) as driver:
    # Get rotation from target_frame to j2000
    j2000 = driver.frame_chain
    target_frame = j2000.find_child_frame(driver.target_frame_id)
    rotation = target_frame.rotation_to(j2000)
    
    # Apply rotation to sensor position and velocity
    j2000RotationPos = rotation._rots.apply(driver.sensor_position[0])
    j2000RotationVel = rotation._rots.apply(driver.sensor_position[1])

    # pass driver instance into formatter function
    usgscsmString = to_usgscsm(driver)

In [ ]:
# load the json encoded string ISD
usgscsm_dict = json.loads(usgscsmString)

# strip the image file extension and append .json 
jsonFile = os.path.splitext(fileName)[0] + '.json'

# write to disk 
with open(jsonFile, 'w') as fp:
    json.dump(usgscsm_dict, fp)

In [ ]:
# Constructs a camera model using usgscsm
model="USGS_ASTRO_FRAME_SENSOR_MODEL"  # Make sure this matches your camera model
plugin = csmapi.Plugin.getList()[0]
isd = csmapi.Isd(fileName)
warns = csmapi.WarningList()
if plugin.canModelBeConstructedFromISD(isd, model, warns):
    print("CONSTRUCTED CAMERA")
    camera = plugin.constructModelFromISD(isd, model)
else:
    print("CAN'T CONSTRUCT CAMERA")
    for item in warns:
        print(item.getMessage())

In [ ]:
# Ingest image and spiceinit it
cube = os.path.splitext(fileName)[0] + '.cub'
isis.mdis2isis(from_=fileName, to=cube)
isis.spiceinit(from_=cube, shape='ellipsoid')

In [ ]:
# Grab campt output on spiceinit'd cube and load it as a pvl
output = isis.campt(from_=cube)
pvl_output = pvl.loads(output)

In [ ]:
# Grab body fixed coordinates from campt pvl output
campt_bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
campt_bodyfixed = np.asarray(campt_bodyfixed.value) * 1000

# Grab body fixed coordinates from csm
isd_bodyfixed = csm.generate_ground_point(0, (256 - .5, 256 - .5), camera)
isd_bodyfixed = np.array([isd_bodyfixed.x, isd_bodyfixed.y, isd_bodyfixed.z])

# Compare the two body fixed coordinates
isd_bodyfixed - campt_bodyfixed

In [ ]:
# Grab sensor position from isd
isd_position = usgscsm_dict['sensor_position']['positions']
isd_position = np.asarray(isd_position)

# Grab spacecraft position from campt pvl output
campt_position = pvl_output['GroundPoint']['SpacecraftPosition']
campt_position = np.asarray(campt_position.value) * 1000

# Compare the two positions
isd_position - campt_position

In [ ]:
# Grab InstrumentPosition table from the isis cube using tabledump
instrument_pos_table = str(isis.tabledump(from_=cube, name='InstrumentPosition'))
parsed_string = instrument_pos_table.split(',')

In [ ]:
# Grab sensor position from the table dump output
isisj2000pos = np.asarray([float(parsed_string[6][4:]), float(parsed_string[7]), float(parsed_string[8])]) * 1000

# Grab ALE's sensor position
isdj2000pos = np.asarray(j2000RotationPos)

# Compare the two sensor positions that are in the j2000 reference frame
isdj2000pos - isisj2000pos

In [ ]:
# Grab velocities from the table dump output
isisj2000vel = np.asarray([float(parsed_string[9]), float(parsed_string[10]), float(parsed_string[11])]) * 1000

# Grab ALE's velocities
isdj2000vel = np.asarray(j2000RotationVel)

# Compare the two velocity lists that are in the j2000 reference frame
isdj2000vel - isisj2000vel

In [ ]:
# Grab spacecraft position and body fixed look vector from csm
locus = camera.imageToRemoteImagingLocus(csmapi.ImageCoord(256 - .5, 256 - .5))
csm_bodyfixedLV = np.asarray([locus.direction.x, locus.direction.y, locus.direction.z])
csm_position = np.asarray([locus.point.x, locus.point.y, locus.point.z])

# Grab spacecraft position and body fixed look vector from campt pvl output
campt_bodyfixedLV = np.asarray(pvl_output['GroundPoint']['LookDirectionBodyFixed'])
campt_position = pvl_output['GroundPoint']['SpacecraftPosition']
campt_position = np.asarray(campt_position.value) * 1000

# Compute the differences
print(csm_bodyfixedLV - campt_bodyfixedLV)
print(csm_position - campt_position)